# __Génétique Humaine Méthodologie__

## **TP: Étude d'association pan-génomique dans le diabète de type 1:**

*Claire Vandiedonck et Fabien Fauchereau - 2021*

>`Contenu de la séance n°1:`
>
> 0. Synopsis
> 1. Étape 1.</u> Évaluation de la faisabilité de l'étude:
>      <br>1.a. Identification des régions déjà connues pour leur association au DT1 de manière significative
>      <br>1.b. Calcul de puissance pour retrouver les mêmes régions ou d’autres variants avec effets similaires dans une nouvelle cohorte    
> 2. Étape 2. Évaluation de la faisabilité de l'étude
>      <br>2.a. Formatage des fichiers pour les logiciels de génétique – Apprentissage de PLINK (séance 1)
>
>    ...suite dans `T1D_GWA_sesions2and3.ipynb`



---
## **Avant d'aller plus loin**

<div class="alert alert-block alert-danger"><b>Caution:</b> 
Ne travaillez pas directement sur ce notebook. Dupliquez-le et renommez-le par exemple avec vos initiales et travaillez sur cette nouvelle copie : dans le panneau de gauche, faites un clic droit sur le fichier et sélectionnez "Duplicate"<br>
N'oubliez pas de sauvegarder régulièremeny votre notebook.
</div>

<div class="alert alert-block alert-info"> 
    
<b>Rappel</b>, dans un notebook :

- la combinaison de touches <kbd>Ctrl</kbd>+<kbd>Entrée</kbd> exécute une cellule.
- la combinaison de touches <kbd>Shift</kbd>+<kbd>Entrée</kbd> exécute une cellule puis passe à la suivante. C'est équivalent à cliquer sur l'icone ▶️ dans la barre de menu du notebook.
- la combinaison de touches <kbd>Alt</kbd>+<kbd>Entrée</kbd> exécute une cellule puis en crée une nouvelle (vide) en dessous.
- pour ajouter une cellule, vous pouvez aussi cliquer sur l'icone ➕ dans la barre de menu du notebook.
- vous pouvez déplacer les cellules pour les réorganiser les unes en-dessous des autres.
- vous pouvez ajouter des commentaires, soit en commençant la ligne par un "#" dans une cellule de code (ces llognes ne seront pas éxécutées), soit dans une nouvelle déllule de type Markdown.
- vous sélectionner le type de cellule dans le menu en hait de votre notebbok.
    - "Code" pour entrer des lignes de commande à éxécuter 
    - "Markdown pour ajouter du texte balisé qui peut être formatté
- pour modifier une cellule ̀Markdown, double-cliquez dessus
- attention à la casse des caractères pour les cellules de code
- évitez les acartères spéciaux dans les cellules de code

    

 </div>    

***Useful links***    
*To make nice html reports with markdown:* [html visualization tool1](https://dillinger.io/) or [html visualization tool2](https://stackedit.io/app#) and [to draw nice tables](https://www.tablesgenerator.com/markdown_tables ) and the [Ultimate guide](https://medium.com/analytics-vidhya/the-ultimate-markdown-guide-for-jupyter-notebook-d5e5abf728fd)   
*Further reading on JupyterLab notebooks:*  https://jupyterlab.readthedocs.io/en/latest/user/notebook.html <br>
*Here we are using JupyterLab interface implemented as part of the https://plasmabio.org/ project lead by Sandrine Caburet, Pierre Poulain and Claire Vandiedonck.*


---


__*=> A propos de ce jupyter notebook*__

Pour ce TP, nous aurons besoin d'éxécuter du code `bash` at `R`. On pourrait réaliser le TP directement dans un terminal `bash` en lançant R à la volée.

>_If you want to see this by yourself, you can open a terminal on adenine:_
>- _in the `File` menu in the top bar, select `New Launcher` or click on the `+` sign below_
>- _open either a bash `Console` or a `Terminal`_
>- _you'll be able to copy and paste the commands from the `Code` cells of the notebook in the "bottom cell" (for the console) or after the `$` sign (for the terminal)_
>
>_This is for your information only, and not needed. All the commands are already included in this notebook_

Toutefois, nous préférons utiliser un notebook pour rédiger un rapport computationel. Or un notebook jupyter ne peut contenir qu'un seul langage à la fois. Nous avions trois solutions pour éxécuter malgré tout des commandes bash et R dans le même notebook:

**1.** utiliser un notebokk avec un noyau  `bash` et éxécuter des commandes `R`, une par une, en tapant `R -e "myRcommand"` ou éxcéuter un script R contenant plusieurs commandes en tapant `Rscript myscript.R`.

**2.** utiliser un notebook avec un noyau `R` et éxécuter des commandes `bash` en invoquant la function `system("mybashcommand")`.

**3.** utiliser un notebook avec un noyau `python` et indiquer que les cellules doivent être éxécutées en bash (natif) en ajoutant `%%bash` au début de chaque cellule, et en ajoutant `%%R` au début de chaque cellule `R` après avoir installé et chargé `Rpy2` au début du notebook.

=> <mark> Nous avons opté pour cette 3ème option avec un jupyter notebook de type  python </mark> qui a l'avantage de ne pas alourdir les commandes `bash` et `R` et d'être très lisible. Les informations de configuration de ce notebbok et de l'environnement jupyterlab associé sont disponibles à ce lien: https://github.com/CVandiedonck/HumanGeneticsMethodology_GWAS 

Avant de commencer, nous devons donc charger le module `Rpy2` avec la commande suivante.

In [ ]:
%load_ext rpy2.ipython

Vous pouvez éxécuter cette commande python pour connaître votre répertoire de travail puis lister ce qu'il contient.

In [ ]:
import os 
os.getcwd()

<div class="alert alert-block alert-warning"><b>Le résultat devrait ressembler à :</b>"/srv/home/mylogin/m1meg_ghm_gwas" avec votre login. Si ce n'est pas le cas, changez de répertoire avec la commande "os.chdir('path')"</div>

Pour lister ce qu'il contient, vous pouvez tenter avec les 3 langages:
- en python:

In [ ]:
os.listdir()

- en bash:

In [ ]:
%%bash
ls

- en R:

In [ ]:
%%R
list.files()

---
## **0 - Synopsis**

### **Objectifs du TP**

Dans cette série de 4 séances du TP n°1, vous allez réaliser une étude d’association génétique dans le diabète de type 1 (DT1) dans une cohorte française au moyen d’outils informatiques et statistiques. Tous les échantillons ont été génotypés au moyen de la puce Illumina Immunochip (Trynka et al., 2011, Parkes M et al. 2013). Au cours de ce TP, vous serez amenés à préparer vos fichiers de génotypage pour l'analyse, à effectuer des contrôles de qualité et à nettoyer/filtrer les données, à effectuer les analyses d'associations génétiques de type cas/contrôles et de type familial, et enfin à interpréter vos résultats.


### **Plan du TP**

Le découpage des différentes étapes est ainsi prévu :

#### <b><u>Étape n°1 :</u> Évaluation de la faisabilité de l'étude *(pendant la séance de TP 1)</b>

- Identification des régions déjà connues pour leur association au DT1 de manière significative

- Calcul de puissance pour retrouver les mêmes régions ou d’autres variants avec effets similaires dans une nouvelle cohorte
      
#### <b><u>Étape n°2:</u> Étude d’association cas-contrôles *(pendant les séances de TP 1, 2, 3)</b>

- Formatage des fichiers pour les logiciels de génétique – Apprentissage de PLINK (séance 1)

- Statistiques descriptives des cas et contrôles, contrôle qualité et nettoyage des données pour l'étude d'association (séance 2)

- Analyse d'association génétique cas-contrôles (séances 2 et 3)
      
#### <b><u>Étape n°3:</u> Étude d’association familiale *(pendant la séance de TP  4)</b>
- Contrôle de qualité des données familiales

- Analyse d'association génétique familiale

- Représentation des associations

- Interprétation des données, conclusion

 
⚠️ <mark>Vous aurez 3 notebooks: un pour la séance 1, 1 pour les séances 2 et 3 et un dernier pour la séance 4.</mark>


***Références***
Trynka G., et al. Dense genotyping identifies and localizes multiple common and rare variant association signals in celiac disease. Nature Genetics 43, 1193–1201 (2011)
Parkes M, et al. Genetic insight into common pathways and complex relationships among immune-mediated diseases. Nature Genetics 14, 661-673 (2013)


___
---
## **I - Étape 1: Évaluation de la faisabilité de l'étude**

Avant d'entamer toute étude de génétique, il est nécessaire d'évaluer l'état des connaissances et de vérifier si l'étude prévue aura la puissance nécessaire pour identifier des associations génétiques.


### **1- Quelles sont les régions génétiques déjà associées au DT1 ?**

Pour répondre à cette question, vous allez étudier le résultats publiés par par Onengut-Gumuscu et al. Nature Genetics (2015), présentant les derniers résultats de GWAS publiés par le consortium international de l'étude du diabète de type 1 (T1DGC for Type 1 Diabetes genetics Consortium). L'article au format .pdf est présent sur moodle et dans votre environnement jupyterlab.

#### **Q1.1:** Combien de régions génomiques avaient été impliquées dans le diabète de type I (DT1) avant le démarrage de l'étude ?

*votre réponse...*

#### **Q1.2:** Dans quels buts a été conçue l'Immunochip ? Quelle a été la stratégie de conception de l'Immunochip ?

v*otre réponse...*

#### **Q1.3:** Quelle est la structure des échantillons? Combien de cas et de contrôles ont été étudiés après QC (Quality Controls sur les SNPs et sur les sujets) ? Quelle est leur origine géographique?

*votre réponse...*

#### **Q1.4:** Combien de SNPs ont passé les seuils de QC ? 

*votre réponse*

#### **Q1.5:** Avec R, calculer la valeur de seuil de Bonferroni pour le nombre de SNPs ayant passé les QCs. S'agit-il du seuil utilisé dans l'artcile? 

In [ ]:
%%R
# votre code R

#### **Q1.6:** En dehors du CMH qui n'est pas étudié dans cet article, combien de régions ont été trouvées associées à leur seuil? En bas de la colonne 2 de la 1ère page de l'article, combien de régions les auteurs répliquent-ils et combien de nouvelles régions sont associées au seuil 5e-8 ?

*votre réponse*

#### **Q1.7:** Quelle version de la séquence de référence du génome est utilisée dans cet article?

*votre réponse*

<div class="alert alert-block alert-warning"><b>Pour information</b><br>
L'ensemble des études pangénomiques sur le DT1 réalisées à ce jour a permis d'identifier 52 régions associées de manière significative au seuil pangénomique 5x10-8.. Cinq autres régions sont associées de manière suggestives au seuil 5x10-5  : 2q11.2, 6q23.3, 11q13.1, 14q24.1 et 17q21.31, les 4 premières étant associées de manière significative à une autre maladie autoimmune ou inflammatoire.</div>

### **2- Quelle est la puissance de détecter les mêmes associations ou de nouvelles associations avec des effets comparables ?**

Dans cette seconde partie, vous allez évaluer la capacité de répliquer les effets déjà publiés dans une cohorte française génotypée au moyen de l'immunochip. Vous réaliserez le GWAS de cette cohorte dans les prochaines séances de TP.

Pour rappel, dans un test d'association génétique, `l'hypothèse nulle (H0)` correspond à l'absence d'association, c'est à dire qu'il n'y a pas d'effet: les fréquences alléliques sont identiques entre cas et contrôles.

`L'hypothèse alternative (H1)` est celle de l'absence d'égalité des fréquences alléliques: il existe un effet, c'est à dire une association allélique.

En statistiques, on définit la `puissance` comme la probabilité de detecter un effet s'il existe. Il s'agit de la probabilité de rejeter H0 sachant que H0 n'est pas vraie. Il faut connaitre une hypothèse H1 spécifique, c'est-à-dire connaitre l'amplitude de l'effet en question. Dans une étude d'association génétique, l'OR correspond à l'effet. 
Dans le cas d'un test de comparaison de fréquences, il faut également connaître la fréquence de chaque groupe, cas et contrôles sous H1. Connaissant la fréquence de contrôles et l'OR, vous pouvez déduire la fréquence des cas en admettant des effectifs de taille similaire entre cas et contrôles (voir formule plus bas).

Si vous ne génotypez pas directement le SNP reporté comme étant associé, mais un SNP en déséquilibre de liaison (DL) avec lui, certains outils de calcul de puissance vous demandent de renseigner ce DL. Dans ce TP, nous espérons que le variant associé est bien inclus dans l'immunochip. Nous supposerons donc que nous testons directement le variant associé pour les calculs de puissance.

=> Vous allez calculer la puissance pour une taille d'échantillon donnée que vous ferez varier de 50 à 5000 (pour chaque groupe, cas et contrôles) pour les deux meilleures régions associées au DT1 à ce jour après le CMH:
- la région du gène de l'Insuline (*INS*) en 11p15.5:  chr11:2092701-2260001
- le gène *PTPN22* en 1p13.2: chr1:113288123-114009223

A titre de référence et pour comparaison, nous vous fournissons les valeurs pour la région du CMH (en 6p21.3 : chr6:32350867-32714887) et les résultats du calcul de puissance dans les tableaux 1 et 2 ci-dessous que vous complèterez. Vou sopuvez aussi compléter les tableaux sur le document google partagé à ce line: https://docs.google.com/spreadsheets/d/1M6C4cbVxKXgWMUIWA10pisS6AXS4RJHzZRUBlP2gij0/edit?usp=sharing.

<center><b>Tableau 1.</b> Valeurs de référence pour les top SNPs des 3 meilleurs régions associées au DT1</center>

|  			région 		 	|  			SNP 		       	|  			allèles 		 	|  			p-value 		 	|  			OR 			 allèle  mineur 		 	|  			OR 			allèle  majeur 		 	|  			MAF 		    	|  			freq 			allèle majeur 		 	|  			freq 			allèle mineur patients 		                	|  			freq 			allèle majeur patients 		 	|
|----------	|-------------	|-----------	|-----------	|----------------------	|---------------------	|----------	|----------------------	|----------------------------------------------	|-------------------------------	|
|  			CMH 		    	|  			rs6916742 		 	|  			C>T 		     	|  			<br>  			 		  	|  			0.24 		               	|  			4.17 		              	|  			0.39 		   	|  			0.61 		               	|  			0.13<br> 			<br> 			=0.39*0.24(1-0.39+0.39*0.24) 		 	|  			0.87 		                        	|
|  			INS 		    	|  			<br>  			 		    	|  			<br>  			 		  	|  			<br>  			 		  	|  			<br>  			 		             	|  			<br>  			 		            	|  			<br>  			 		 	|  			<br>  			 		             	|  			<br>  			 		                                     	|  			<br>  			 		                      	|
|  			PTPN22 		 	|  			<br>  			 		    	|  			<br>  			 		  	|  			<br>  			 		  	|  			<br>  			 		             	|  			<br>  			 		            	|  			<br>  			 		 	|  			<br>  			 		             	|  			<br>  			 		                                     	|  			<br>  			 		                      	|

Pour les deux régions, complétez les 8 1ères colonnes du Tableau 1 à partir des valeurs de l'article Onengut-Gumuscu et al. 2015 :
- le top SNP (celui qui a la p-value la plus faible) et ses allèles Majeur>Mineur
- la p-value de l'association au DT1
- l'OR de risque correspondant
- la fréquence de l'allèle de risque chez les contrôles de l'étude

*entrez ici d'éventuelles explications pour compléter le tableau*

A présent vous pouvez calculer la puissance de détecter ces mêmes effets ou des effets d'amplitude similaire, connaissant la fréquence de l'allèle de risque associé et l'OR.
Il existe un outil en ligne vous permettant de faire ces calculs de puissance : http://osse.bii.a-star.edu.sg/calculation2.php. Attention, toutes les fréquences et probabilités sur ce site sont à renseigner en pourcentage!

Vous devrez indiquer:
- la taille des échantillons
- le degré de signification (vous utiliserez le seuil de p-value corrigé pour un GWAS, c'est-à-dire 5x10-8 = 5x10-6 %)
- la fréquence allélique chez les cas (p)
- la fréquence allélique chez les contrôles (q).

En faisant l'hypothèse que les effectifs de cas et de contrôles sont les mêmes, il est possible de calculer la fréquence allélique p chez les cas à partir de la fréquence de l'allèle q chez les contrôles et à partir de l'OR en utilisant la formule suivante (attention à bien prendre l'OR correspondant à l'allèle de fréquence p et q):</p> 
						
> p = (q x OR) / [1 - q  + (q x OR)]


#### **Q1.8:** Complétez les dernières colonnes du Tableau 1 puis déterminer la puissance pour les 3 régions pour des cohortes cas/contrôle de taille par groupe = 50, 100, 200, 300, 500, 1000, 2000, 3000 et 5000 sujets dans le Tableau 2. Commentez.

<center><b>Tableau 2.</b> Puissance pour les différents effectifs</center>

|  			région 		 	|  			50 		     	|  			100 		    	|  			200 		    	|  			300 		    	|  			500 		    	|  			1000 		   	|  			2000 		   	|  			3000 		   	|  			5000 		   	|
|----------	|----------	|----------	|----------	|----------	|----------	|----------	|----------	|----------	|----------	|
|  			CMH 		    	|  			0.5% 		   	|  			9.4% 		   	|  			69.1% 		  	|  			97.1% 		  	|  			100% 		   	|  			100% 		   	|  			100% 		   	|  			100% 		   	|  			100% 		   	|
|  			INS 		    	|  			<br>  			 		 	|  			<br>  			 		 	|  			<br>  			 		 	|  			<br>  			 		 	|  			<br>  			 		 	|  			<br>  			 		 	|  			<br>  			 		 	|  			<br>  			 		 	|  			<br>  			 		 	|
|  			PTPN22 		 	|  			<br>  			 		 	|  			<br>  			 		 	|  			<br>  			 		 	|  			<br>  			 		 	|  			<br>  			 		 	|  			<br>  			 		 	|  			<br>  			 		 	|  			<br>  			 		 	|  			<br>  			 		 	|
|          	|          	|          	|          	|          	|          	|          	|          	|          	|          	|

*votre commentaire (vous pouvez ajouter des figures)*

<div class="alert alert-block alert-warning"><b>En pratique</b><br>
⚠️  On calcule la puissance avant tout projet afin de connaitre la taille des échantillons recquise pour mettre en évidence un effet minimum. Cependant, on recalcule la puissance à la fin de l'étude en fonction des fréquences alléliques effectivement estimées dans notre échantillon afin de s'assurer, en cas d'absence d'association, qu'on avait bien la puissance nécessaire. Pensez-y pour votre compte-rendu!</div>

---
## ***Références***

- Trynka G., et al. Dense genotyping identifies and localizes multiple common and rare variant association signals in celiac disease. Nature Genetics 43, 1193–1201 (2011)
- Parkes M, et al. Genetic insight into common pathways and complex relationships among immune-mediated diseases. Nature Genetics 14, 661-673 (2013)
- Onengut-Gumuscu S., et al. Fine mapping of type 1 diabetes susceptibility loci and evidence for colocalization of causal variants with lymphoid gene enhancers. Nature Genetics 15, 381-386 (2015)

-> Pour information, voici d'autres outils populaires de calcul de puissance :
<br>Genetic Power Calculator : http://zzz.bwh.harvard.edu/gpc/
<br>Quanto : http://biostats.usc.edu/Quanto.html
<br>R et le package pwr : http://cran.r-project.org/web/packages/pwr/index.html

---
---

## **II - Étape 2: Étude d'association cas-contrôles**

Cette étape sera réalisée au cours des séances de TP n°1 à n°3. L'objectif est de réaliser une étude d'association généique dans le diabète de type 1 (DT1) dans une cohorte française au moyen d'outils bioinformatiques et de statistique génétique. Les sujets d'étude proviennent d'une série de patients atteints de diabète de type 1 et de contrôles sains. Tous les échantillons ont été génotypés au moyen de la puce Illumina immunochip (Trynka et al., 2011, Parekes M. et al., 2013). Au cours de ces deux séances de TP, vous serez amenés:
- à formater vos fichiers de génotypage pour les logiciels d'analyse génétique
- à effectuer des contrôles de qualité
- à nettoyer/filtrer les données
- puis à réaliser l'étude d'association cas-contrôles pangénomique.


### <b><u>Première partie :</u> Formatage des fichiers pour les logiciels de génétique – Apprentissage de PLINK</b>

Pour réaliser l'étude d'association, vous devrez préparer les fichiers sous un format utilisable par les logiciels de génétique. Le principal logiciel que vous utiliserez est PLINK: http://zzz.bwh.harvard.edu/plink/.

En plus de permettre des analyses d'association, ce logiciel permet de manier les fichiers de génétique, de faire des sélections et filtrages, et de réaliser certains contrôles de qualité. Il est mondialement reconnu comme un des meilleurs outils pour réaliser des GWAS.
Ce logiciel libre existe en versions windows, mac et unix.
Le tutoriel (http://zzz.bwh.harvard.edu/plink/tutorial.shtml) est très détaillé et donne de nombreux exemples.

Rendez-vous sur la page d'introduction de PLINK. Lisez les premiers paragraphes.


#### **Q2.1.** Quels sont les usages premiers de ce logiciel? Quelles étapes de l'ensemble du processus d'une étude génétique ne sont pas couvertes par ce logiciel?

#### **Q2.2.** Qui est l'auteur de ce logiciel? A quel laboratoire était-il affilié?

#### **Q2.3.** Quelle est la version actuelle originale de PLINK?

Il existe une nouvelle version récente pour les « experts » ou beta-testeurs. Ouvrez le lien "available" en haut à droite dans une autre fenêtre.

#### **Q2.4.** Quel est le numéro de cette version pour beta-testeurs? Dans quel but a-t-elle été développée ?

Ouvrez le lien "https://www.cog-genomics.org/plink2". Cliquez sur "Credits" à la fin du 1er paragrpahe.

#### **Q2.5.**  Par qui la version 1.9 est-elle principalement développée? Dans quels instituts sont-ils ?

> Dans ce TP, vous utiliserez la version **PLINK 1.9** (https://www.cog-genomics.org/plink2) désormais stable qui a été installée sur adénine. Pour note, il existe une version PLINK2.0 en cours de développement.

<div class="alert alert-block alert-warning"><b>En pratique</b><br>PLINK s'utilise en lignes de commandes « relativement simples ». 
Toutes les informations pour spécifier les fichiers d'entrée (« input »), les fichiers de sortie (« output ») et les paramètres d'utilisation sont détaillés dans le menu à gauche de la page d’accueil de PLINK. Dans la <a href="http://zzz.bwh.harvard.edu/plink/reference.shtml">section 4</a> sont listées toutes les commandes existantes.</div>

Commençons par nous familiariser avec son utilisation avec des fichiers « test » :

#### **-> Lancement de PLINK :**

• Pour faire tourner PLINK sous Unix ou dans ce notebook, vous appelez d'abord le programme en tapant : `plink`

In [ ]:
%%bash
plink

Le programme démarre et s'arrête avec une erreur car vous n'avez pas encore donné de fichiers inputs.

> Si vous travaillez avec PLINK 1.07 au lieu de 1.9, PLINK va essayer de se connecter au réseau pour vérifier la version. Si vous voulez l'éviter, ajouter l'argument:`--noweb`

`plink --noweb`

>Si toutefois vous vouliez utiliser PLINK en dehors d'adenine, voici des conseils d'utilisation **sous Windows**:<br>
>Vous devez utiliser le DOS en utilisant l'invite de commande (cmd.exe) ou, sur les dernières versions de windows, le "windows power shell" à partir du menu "fichier" de votre répertoire de travail.  Vos fichiers "inputs" doivent être placés dans le même répertoire (=dossier) que l'éxécutable de PLINK plink.exe.
>Trois options s'offrent à vous, de la plus compliquée à la plus simple!:
>
>(1) Dans le menu « Démarrer » de Windows, rechercher « cmd » puis indiquez le chemin du répertoire où est situé plink.exe (pour changer de répertoire, la commande est “cd”; pour passer au répertoire supérieur, taper “cd ..” ; vous pouvez utiliser la flèche de tabulation à gauche du clavier pour autocompléter les noms de dossiers)
> 
>(2) Alternativement, trouvez le fichier cmd.exe (en général dans Windows\system 32; vous pouvez le trouver aussi dans les accessoires). Faire une copie que vous placez dans le dossier contenant plink.exe. Si vous double-cliquez alors sur cmd.exe dans ce répertoire, vous pouvez utiliser PLINK.
>
>(3) Encore plus simplement, vous pouvez aussi créer en local un répertoire dans lequel vous mettrez tous vos documents du TP (fichiers d'input, d'output) ainsi que vos programmes. Copiez et collez l'executable plink.exe dans ce répertoire. Copiez et collez également l'exécutable de l'invite de commande cmd.exe dans ce répertoire. Si vous double-cliquez alors sur cmd.exe dans ce répertoire, vous pouvez utiliser PLINK.
>
>Alternativement, si vous travaillez depuis windows power shell, il faut spécifier que plink est dans le répertoire de travail en tapant: .\plink
      

#### **-> Spécification des noms de fichier de sortie :**

A chaque commande PLINK, si vou spécifiez des arguments (vrai aussi sans argument pour PLINK 1.07), des fichiers de sortie (outputs) sont automatiquement générés avec différentes extensions. Vous pouvez spécifier le préfixe de ces outputs avec l'argument `-- out`.
En particulier, vous obtenez un fichier .log qui a sauvegardé tout ce qui s'était affiché dans votre terminal ou dans le notebook.

Essayons en utilisant le préfixe "test":

In [ ]:
%%bash
plink --out  test

Vous obtenez un fichier texte `test.log` que vous pouvez ouvrir. Notez qu'il contient une ligne après « Options in effect » : « --out test »

Dans PLINK, vous pouvez entrer plusieurs arguments à la suite sur la même ligne de commande. Si vous avez utilisé l'argument précédent, tous les autres outputs générés contiendront le même préfixe « test » suivis de leur extension propre.

#### **-> Fichiers d'entrée :**

PLINK a besoin de deux types de fichiers d'entrée.

#### **Q2.6.**  Quels sont les extensions de ces fichiers ? A quoi correspondent-ils ?

Dans le répertoire `input` de votre environnement jupyterlab, vous avez des fichiers d'entrée d'exemple avec le préfixe « test ». Les fichiers sont aussi disponibles sur moodle.

Ouvrez-les en double cliquant dessus dans le jupyterlab (ou avec un éditeur de texte comme notepad++ sous windows ou gedit sous unix si vous ne travaillez pas sur adénine).
Cherchez des explications sur leur structure dans la documentation de PLINK correspondante dans la section "5. Basic usage/data formats" (http://zzz.bwh.harvard.edu/plink/data.shtml)  du menu.

#### **Q2.7.**  Concernant le fichier de structure de pedigree:

      - Combien de sujets ont été génotypés ?

    - Sont-ils apparentés ? Ont-ils les mêmes parents ? Quelles colonnes vous donnent ces informations ?

    - Quel est l'identifiant du 5ème sujet ? Est-il possible de le distinguer des autres sujets en considérant un identifiant unique ?

    - Quel est le sexe des sujets ? Quelle colonne vous donne cette information ? Est-ce cohérent avec les autres colonnes ?

    - Connaît-on leur statut vis-à-vis de la maladie ? Combien y a-t-il de cas et de contrôles ?

    - Combien de marqueurs ont été génotypés pour ces sujets ?

    - Combien de colonnes contiendraient un fichier de pedigree avec n marqueurs génotypés ?

#### **Q2.8.**  Concernant le fichier de marqueurs :
    
       - Combien y-a-t-il de marqueurs dans ce fichier ? Est-ce en accord avec le fichier de pedigree ?

    - Sur quels chromosomes sont-ils situés ?

    - Connaissez-vous leur position physique ? A partir de quelle origine ?

    - Connaissez-vous leur position génétique ?

<br>
Chargez à présent les fichiers d'input « test » sans les modifier dans PLINK. Il existe deux commandes possibles.

#### **Q2.9.**  Quelles sont ces deux commandes  ? Pouvez-vous lire le fichier de pedigree seul ? Quel problème rencontrez-vous ?

In [ ]:
%%bash
#votre premiere commande

In [ ]:
%%bash
#votre deuxième commande

On apprend désormais que le fichier de pedigree ne contenait les génotypes que de marqueurs autosomiques. Nettoyez le fichier de marqueurs avec un éditeur de texte et sauvegardez le sous un autre nom (ex : `autosomic.map`) et chargez le dans PLINK avec les fichier de pedigree. N'oubliez-pas de changer le préfixe des sorties pour ne pas écraser le précédent fichier .log.

In [ ]:
%%bash
#votre commande

#### **Q2.10.** Les informations obtenues sont-elles désormais cohérentes avec ce que vous savez?

Cette structure de pedigree et de liste de marqueurs est utilisée par la plupart des logiciels de génétique. Il existe cependant quelques variantes, en particulier pour l'encodage des phénotypes parfois dans un troisième fichier, ou pour le fichier de marqueurs qui ne contient pas toujours la position.
**Cependant, l'ordre des colonnes des génotypes dans le fichier de pedigree doit toujours correspondre à l'ordre des marqueurs donnés. Il est donc fondamental de bien formater ses fichiers !**

PLINK peut travailler avec des formats de fichier de grande taille : nombreux sujets et nombreux génotypes.
Dans ce cas, il existe une version binaire des fichiers moins gourmande en espace disque et en temps d'analyse. Consultez la section « Binary PED files » (http://zzz.bwh.harvard.edu/plink/data.shtml#bed) et effectuez la conversion des fichiers test en format binaire en sauvegardant les outputs avec le préfixe `mytest_bdata`.


#### **Q2.11.**Concernant ce format binaire :
       - Quelle commande utilisez-vous pour obtenir cette conversion ?


In [ ]:
%%bash
#votre commande

       - Quel fichier est converti en format binaire ? Quelle est l'extension de ce fichier ?

       - Quels autres fichiers ont été générés ? A quoi correspondent-ils ?

       - Quelle information n'était pas dans le fichier .map et est désormais présente dans le fichier de marqueurs ?

       - Quelle commande utilisez-vous pour lire ces fichiers dans plink ?

In [ ]:
%%bash

Il existe un moyen de convertir des données binaires en format non binaire.
Allez dans la section 6. Data management (http://zzz.bwh.harvard.edu/plink/dataman.shtml) et identifier la commande.

#### **Q2.12.**Quelle commande utilisez-vous pour recoder au format ped/map ?

In [ ]:
%%bash
# votre commande

#### **Q2.13.**Quelle différence observez vous entre le nouveau fichier .ped et le fichier .ped initial ?

<div class="alert alert-block alert-success"><b>=> Bravo!</b><br>

Vous êtes désormais prêts pour travailler avec les fichiers de cas et de contrôles.</div>

---
---
Suite du TP dans le prochain notebook!